In [5]:
import pygame
import random

pygame.init()

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 800
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Intersection Simulation")

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)

class Car:
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.speed = 2
        self.width = 20
        self.length = 40
        self.stopped = False

    def move(self):
        if not self.stopped:
            if self.direction == 'up':
                self.y -= self.speed
            elif self.direction == 'down':
                self.y += self.speed
            elif self.direction == 'left':
                self.x -= self.speed
            elif self.direction == 'right':
                self.x += self.speed

    def draw(self, win):
        if self.direction == 'up':
            pygame.draw.rect(win, RED, (self.x, self.y, self.width, self.length))
        elif self.direction == 'down':
            pygame.draw.rect(win, RED, (self.x - self.width, self.y, self.width, self.length))
        elif self.direction == 'left':
            pygame.draw.rect(win, BLUE, (self.x, self.y - self.width, self.length, self.width))
        else:
            pygame.draw.rect(win, BLUE, (self.x, self.y, self.length, self.width))

    def stop(self):
        self.stopped = True

    def can_move(self, other_cars, stoplight):
        if (self.direction in ['up', 'down'] and stoplight == 1) or (self.direction in ['left', 'right'] and stoplight == 0):
            return False
        for other_car in other_cars:
            if other_car.stopped:
                if self.direction == other_car.direction == 'up':
                    if self.y < other_car.y + other_car.length + 5:
                        return False
                elif self.direction == other_car.direction == 'down':
                    if self.y > other_car.y - self.length - 5:
                        return False
                elif self.direction == other_car.direction == 'left':
                    if self.x < other_car.x + other_car.length + 5:
                        return False
                elif self.direction == other_car.direction == 'right':
                    if self.x > other_car.x - self.length - 5:
                        return False
        return True

def main():
    run = True
    clock = pygame.time.Clock()

    cars = []
    prev_time = 0
    stoplight = 0

    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        time = pygame.time.get_ticks() // 1000
        if time != prev_time:
            if time % 3 == 0:
                direction = random.choice(['up', 'down', 'left', 'right'])
                if direction == 'up':
                    cars.append(Car(WINDOW_WIDTH//2, WINDOW_HEIGHT, direction))
                elif direction == 'down':
                    cars.append(Car(WINDOW_WIDTH//2, 0, direction))
                elif direction == 'left':
                    cars.append(Car(WINDOW_WIDTH, WINDOW_HEIGHT//2, direction))
                elif direction == 'right':
                    cars.append(Car(0, WINDOW_HEIGHT//2, direction))
            if time % 15 == 0:
                stoplight = 1 - stoplight
            prev_time = time

        for car in cars:
            if not car.stopped:
                if ((car.direction == 'up' and car.y == WINDOW_HEIGHT//2 + 50) or
                    (car.direction == 'down' and car.y + car.length == WINDOW_HEIGHT//2 - 50) or
                    (car.direction == 'left' and car.x == WINDOW_WIDTH//2 + 50) or
                    (car.direction == 'right' and car.x + car.length == WINDOW_WIDTH//2 - 50)) and not car.can_move(cars, stoplight):
                    car.stop()
                else:
                    car.move()
            elif car.stopped and car.can_move(cars, stoplight):
                    car.stopped = False
                    car.move()

            if car.stopped and car.can_move(cars, stoplight):
                car.stopped = False

        cars = [car for car in cars if 0 <= car.x <= WINDOW_WIDTH and 0 <= car.y <= WINDOW_HEIGHT]

        window.fill(WHITE)
        pygame.draw.line(window, BLACK, (0, WINDOW_HEIGHT//2), (WINDOW_WIDTH, WINDOW_HEIGHT//2), 5)
        pygame.draw.line(window, BLACK, (WINDOW_WIDTH//2, 0), (WINDOW_WIDTH//2, WINDOW_HEIGHT), 5)
        if stoplight == 0:
            pygame.draw.circle(window, GREEN, (WINDOW_WIDTH//2 + 30, WINDOW_HEIGHT//2 + 30), 10)
            pygame.draw.circle(window, RED, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30), 10)
        else:
            pygame.draw.circle(window, RED, (WINDOW_WIDTH//2 + 30, WINDOW_HEIGHT//2 + 30), 10)
            pygame.draw.circle(window, GREEN, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30), 10)
        for car in cars:
            car.draw(window)
        pygame.display.update()

    pygame.quit()

if __name__ == "__main__":
    main()
